In [1]:
import baostock as bs
from  baostock.data.resultset import ResultData
import datetime
import pandas as pd
from typing import Optional

class DataProvider():

    def get_stock_list(self)->list:
        raise NotImplementedError
    
    def get_data_handler(self):
        raise NotImplementedError

    def on_exit(self):
        pass
    
class BaoStockDataProvider(DataProvider):

    def __init__(self,
                 latest_trade_date:Optional[str]=None):
        super().__init__()
        self._check_error(bs.login())
        self.latest_trade_date = latest_trade_date
        
    def _check_error(self, result: ResultData):
        if result.error_code != '0':
            raise RuntimeError('error_code: ' + result.error_code + ', error_msg: ' + result.error_msg)
        return result
        
    def on_exit(self):
        self._check_error(bs.logout())
    
    def _collect_all(self,result: ResultData)->list:
        data_list = []
        while (result.error_code == '0') & result.next():
            data_list.append(result.get_row_data())
        return data_list

    def get_stock_list(self):
        stock_list = self._check_error(bs.query_all_stock(day=self._get_latest_trade_date()))
        res= []
        while (stock_list.error_code == '0') & stock_list.next():
            res.append(stock_list.get_row_data()[0])
        return res

    def _get_latest_trade_date(self)->str:
        if self.latest_trade_date is not None:
            return self.latest_trade_date
        current_date = datetime.datetime.now().date()
        start_date = current_date - datetime.timedelta(days=30)
        trade_date = self._check_error(bs.query_trade_dates(start_date=start_date.strftime('%Y-%m-%d'), 
                                                   end_date=current_date.strftime('%Y-%m-%d')))
        for date in reversed(self._collect_all(trade_date)):
            if date[1] == '1':
                return date[0]
        raise RuntimeError('No trade date found')            

In [2]:
bao=BaoStockDataProvider()
print(bao.get_stock_list())
bao.on_exit()

login success!
['sh.000001', 'sh.000002', 'sh.000003', 'sh.000004', 'sh.000005', 'sh.000006', 'sh.000007', 'sh.000008', 'sh.000009', 'sh.000010', 'sh.000011', 'sh.000012', 'sh.000013', 'sh.000015', 'sh.000016', 'sh.000017', 'sh.000018', 'sh.000019', 'sh.000020', 'sh.000021', 'sh.000022', 'sh.000025', 'sh.000026', 'sh.000027', 'sh.000028', 'sh.000029', 'sh.000030', 'sh.000031', 'sh.000032', 'sh.000033', 'sh.000034', 'sh.000035', 'sh.000036', 'sh.000037', 'sh.000038', 'sh.000039', 'sh.000040', 'sh.000041', 'sh.000042', 'sh.000043', 'sh.000044', 'sh.000045', 'sh.000046', 'sh.000047', 'sh.000048', 'sh.000049', 'sh.000050', 'sh.000051', 'sh.000052', 'sh.000053', 'sh.000054', 'sh.000056', 'sh.000057', 'sh.000058', 'sh.000059', 'sh.000060', 'sh.000061', 'sh.000062', 'sh.000063', 'sh.000064', 'sh.000065', 'sh.000066', 'sh.000067', 'sh.000068', 'sh.000069', 'sh.000070', 'sh.000071', 'sh.000072', 'sh.000073', 'sh.000074', 'sh.000075', 'sh.000076', 'sh.000077', 'sh.000078', 'sh.000079', 'sh.00009

AttributeError: 'BaoStockDataProvider' object has no attribute 'check_error'

In [5]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg


def collect_data(frequency="d"):
    rs = bs.query_history_k_data_plus("sh.600000",
    "date,open,close,high,low,volume",
    start_date='2024-07-01', end_date='2024-12-31',
    frequency="d", adjustflag="3")
    price_list = []
    volume_list = []
    while (rs.error_code == '0') & rs.next():
        data=rs.get_row_data()
        price_list.append([data[0],data[1],data[2],data[3],data[4]])
        volume_list.append([data[0],data[5]])
    return pd.DataFrame(price_list,columns=['date','open','close','high','low']),pd.DataFrame(volume_list,columns=['date','volume'])

price,volume=collect_data()
print(price)
print(volume)

#### 登出系统 ####
bs.logout()

login success!
           date     open    close     high      low
0    2024-07-01   8.2200   8.2800   8.3000   8.2000
1    2024-07-02   8.2700   8.5300   8.5400   8.2700
2    2024-07-03   8.5300   8.5300   8.6000   8.4800
3    2024-07-04   8.5900   8.5000   8.6300   8.4800
4    2024-07-05   8.5400   8.3400   8.5500   8.2900
..          ...      ...      ...      ...      ...
120  2024-12-25  10.1300  10.3500  10.3900  10.1300
121  2024-12-26  10.3500  10.3400  10.3900  10.1600
122  2024-12-27  10.3300  10.3600  10.3900  10.1200
123  2024-12-30  10.3400  10.4700  10.4900  10.3100
124  2024-12-31  10.4300  10.2900  10.5600  10.2800

[125 rows x 5 columns]
           date    volume
0    2024-07-01  28732962
1    2024-07-02  57345810
2    2024-07-03  46409668
3    2024-07-04  44412066
4    2024-07-05  51997018
..          ...       ...
120  2024-12-25  72739600
121  2024-12-26  61152236
122  2024-12-27  69506374
123  2024-12-30  78477805
124  2024-12-31  57783443

[125 rows x 2 columns]
l

In [1]:
import baostock as bs
bs.logout()

you don't login.


: 